In [13]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/Users/daniilkorolkov/Desktop/Тинькофф | Поиск Конверсионных тематик/oper_prepared_phrases.csv', sep = ';')
df.head()

,Unnamed: 0,hi_task_id,start_date,end_date,length,phrases,phrases_concated,operator_user_id,wo_hit_status_result_desc,oper_phrases,oper_words,bl,oper_words_stemmed
0,0,12078590742#36469131809,18.12.2021 7:04:16,18.12.2021 7:05:27,71,"0 алло, 1 да, 1 здравствуйте меня зовут * я зв...",1,410131.0,NaN,"1 да, 1 здравствуйте меня зовут * я звоню с пр...","['кредитную', 'лимитом', 'рублей', 'беспроцент...",0,"['кредитн', 'лимит', 'рубл', 'беспроцентн', 'к..."
1,1,12078913588#36450601588,18.12.2021 8:06:12,18.12.2021 8:10:06,234,"0 *, 1 здравствуйте, 0 добрый день, 1 меня зов...",1,136548.0,NaN,"1 здравствуйте, 1 меня зовут * я звоню вам с и...","['предлагаем', 'нашу', 'кредитную', 'лимитом',...",0,"['предлага', 'наш', 'кредитн', 'лимит', 'рубл'..."
2,2,12082226162#36492491475,18.12.2021 6:01:38,18.12.2021 6:03:12,75,1 здравствуйте меня зовут * звоню с интересным...,2,1009562.0,NaN,1 здравствуйте меня зовут * звоню с интересным...,"['предлагаем', 'кредитную', 'лимитом', 'рублей...",0,"['предлага', 'кредитн', 'лимит', 'рубл', 'спец..."
3,3,12082932286#36494587977,18.12.2021 6:00:38,18.12.2021 6:04:59,261,"0 ай, 1 алло, 1 меня * * звоню с целью озвучит...",1,310475.0,NaN,"1 алло, 1 меня * * звоню с целью озвучить для ...","['банком', 'предлагается', 'высоким', 'кредитн...",0,"['банк', 'предлага', 'высок', 'кредитн', 'лими..."
4,4,12083613174#35456293560,18.12.2021 8:14:18,18.12.2021 8:14:56,38,"0 алло, 1 здравствуйте звонок сорвался извинит...",1,457890.0,NaN,1 здравствуйте звонок сорвался извините меня *...,"['банковским', 'предложением', 'спешите', 'отк...",0,"['банковск', 'предложен', 'спеш', 'отказыва', ..."


In [5]:
data = df.query('wo_hit_status_result_desc == ("Дозвон, Отказ", "Дозвон, Успешно")')[['oper_words_stemmed', 'wo_hit_status_result_desc']]

data.wo_hit_status_result_desc = (data.wo_hit_status_result_desc == "Дозвон, Успешно").astype(int)

In [6]:
df_train, df_test = train_test_split(data, test_size=0.2, shuffle=True, random_state=123)

In [7]:
vectorizer = TfidfVectorizer(ngram_range = (2,2), stop_words = ['календарн', 'уг', 'фамил', 'отчеств', 'рожден']).fit(df_train.oper_words_stemmed.astype(str))

In [8]:
X_train_vectors = vectorizer.transform(df_train.oper_words_stemmed.astype(str))
X_test_vectors  = vectorizer.transform(df_test. oper_words_stemmed.astype(str))

In [9]:
clf = MultinomialNB().fit(X_train_vectors, df_train.wo_hit_status_result_desc)

In [10]:
X = vectorizer.transform(data.oper_words_stemmed.astype(str))

In [11]:
predicts = clf.predict(X)
print(classification_report(data.wo_hit_status_result_desc, predicts))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    516289
           1       0.97      0.74      0.84     19376

    accuracy                           0.99    535665
   macro avg       0.98      0.87      0.92    535665
weighted avg       0.99      0.99      0.99    535665



In [14]:
i = 50

neg_class_prob_sorted = clf.feature_log_prob_[0, :].argsort()[::-1]
pos_class_prob_sorted = clf.feature_log_prob_[1, :].argsort()[::-1]

print('Negative\n\nfront:', np.take(vectorizer.get_feature_names(), neg_class_prob_sorted[:i]),'\n')
#print('back:', np.take(vectorizer.get_feature_names(), neg_class_prob_sorted[-i*2::]),'\n')
print('Positive\n\nfront:', np.take(vectorizer.get_feature_names(), pos_class_prob_sorted[:i]),'\n')
#print('back:', np.take(vectorizer.get_feature_names(), pos_class_prob_sorted[-i*2::]))

Negative

front: ['банковск предложен' 'банковск услуг' 'предложен банковск'
 'предлага кредитн' 'кредитн лимит' 'лимит рубл' 'беспроцентн период'
 'рубл беспроцентн' 'период покупк' 'покупк дне' 'дне рассрочк'
 'достав бесплатн' 'люб удобн' 'рассрочк месяц' 'бесплатн люб'
 'удобн оформлен' 'предложен прослуша' 'месяц партнер' 'кредитн карт'
 'магазин партнер' 'предложен добр' 'рассрочк магазин' 'оформлен ответ'
 'партнер достав' 'информац полезн' 'партнер месяц' 'спеш отказыва'
 'предложен предложен' 'предложен информац' 'предлож кредитн'
 'оформлен обязател' 'обязател ответ' 'предложен спеш' 'бесплатн оформлен'
 'предложен кредитн' 'выпуск доставк' 'выгодн услов' 'предложен сфер'
 'полезн дальн' 'услуг прослуша' 'хотел предлож' 'месяц выпуск'
 'выпуст достав' 'кредитн беспроцентн' 'абсолютн бесплатн'
 'предложен услуг' 'выгодн банковск' 'месяц достав' 'месяц выпуст'
 'хот предлож'] 

Positive

front: ['ваш заявк' 'заявк кредитн' 'дня результат' 'лимит дня' 'кредитн лимит'
 'беспроцен